## Load necessary modules

In [1]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import time

# use this to change which GPU to use
gpu = 0

# set the modified tf session as backend in keras
setup_gpu(gpu)

Using TensorFlow backend.
/home/jasper/git/CEIG/keras-retinanet/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jasper/git/CEIG/keras-retinanet/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jasper/git/CEIG/keras-retinanet/venv/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

/home/jasper/git/CEIG/keras-retinanet/venv/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jasper/git/CEIG/keras-retinanet/venv/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jasper/git/CEIG/keras-retinanet/venv/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

## Load RetinaNet model

In [8]:
# adjust this to point to your downloaded/trained model
# models can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
model_path = os.path.join('..', 'DayResults', 'resnet50_pascal_121.h5')

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'plum'}

tracking <tf.Variable 'Variable_15:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_16:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_17:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_18:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_19:0' shape=(9, 4) dtype=float32> anchors


In [9]:
# load images
print("Running tests")
print("If running slow, did CUDA get loaded?")

datasetPath = '/home/jasper/Datasets/Day/'
testPath = datasetPath + 'ImageSets/Main/test.txt'
imgsPath = datasetPath + 'JPEGImages/'

visualise = False

testFile = open(testPath, 'r')

resultPath = os.path.split(model_path)[0]
resultPath = os.path.join(resultPath, 'detections.txt')
resultFile = open(resultPath,'w')

firstStart = time.time()
cont = testFile.readlines()
for i in range(0, len(cont)):
    print("detecting image: {}".format(cont[i].rstrip()))
    
    imgPath = imgsPath + cont[i].rstrip() + '.jpg'
    image = read_image_bgr(imgPath)
    
    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)
    
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    print("processing time: ", time.time() - start)

    # correct for image scale
    boxes /= scale

    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score < 0.01:
            break
                
        bb = np.array(box).astype(int)
        dataline = cont[i].rstrip() + ' ' + str(score) + ' ' + str(bb[0]) + ' ' + str(bb[1]) + ' ' + str(bb[2]) + ' ' + str(bb[3]) + '\n' 
        resultFile.write(dataline)
        #print(dataline)
        
        
        
    if(visualise):
        # visualize detections
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
            # scores are sorted so we can break
            if score < 0.5:
                break

            color = label_color(label)

            b = box.astype(int)
            draw_box(draw, b, color=color)

            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)

        plt.figure(figsize=(15, 15))
        plt.axis('off')
        plt.imshow(draw)
        plt.show()
    
print("DONE~!!")
print("Detected on {} images in {} sec".format(len(cont), time.time() - firstStart))

Running tests
If running slow, did CUDA get loaded?
detecting image: 0004
processing time:  4.537842750549316
detecting image: 0008
processing time:  0.05808854103088379
detecting image: 0012
processing time:  0.06860947608947754
detecting image: 0016
processing time:  0.0684511661529541
detecting image: 0020
processing time:  0.06253361701965332
detecting image: 0024
processing time:  0.06366753578186035
detecting image: 0028
processing time:  0.05338740348815918
detecting image: 0032
processing time:  0.05960798263549805
detecting image: 0036
processing time:  0.0655968189239502
detecting image: 0040
processing time:  0.0607450008392334
detecting image: 0044
processing time:  0.0628807544708252
detecting image: 0048
processing time:  0.05896449089050293
detecting image: 0052
processing time:  0.067169189453125
detecting image: 0056
processing time:  0.062007904052734375
detecting image: 0060
processing time:  0.05279827117919922
detecting image: 0064
processing time:  0.0583527088165